In [27]:
import json
import pandas as pd
from datetime import datetime
import database as db

In [3]:
# Read the JSON file
with open('agri_data.physique.json', 'r') as file:
    data = json.load(file)

In [23]:
df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: datetime.fromisoformat(x['$date'][:10]).date()))
df = df[['Date', 'Produit', 'Place', 'Prix']]
df

,Date,Produit,Place,Prix
0,2021-02-15,Mais,Bordeaux Rendu,210.00
1,2021-02-15,Colza,Rouen Rendu,417.00
2,2021-02-15,Mais,Rhin Fob,227.00
3,2021-02-15,Mais,Bordeaux Fob,215.00
4,2021-02-15,Ble dur,Port-La-Nouvelle Rendu,240.00
...,...,...,...,...
14323,2024-11-26,Ble tendre,Rouen Rendu,215.50
14324,2024-11-26,Mais,Bordeaux Fob,203.00
14325,2024-11-26,Mais,Rhin Fob,206.75
14326,2024-11-26,Ble tendre,Creil Fob,213.00


In [20]:
dfEure = pd.read_csv('dataEureLoire.csv', usecols=['Date', 'Produit', 'Place', 'Prix'])
dfEure['Date'] = pd.to_datetime(dfEure['Date'])

In [ ]:
dfHist = pd.concat([df, dfEure])

,Date,Produit,Place,Prix
0,2021-02-15,Mais,Bordeaux Rendu,210.00
1,2021-02-15,Colza,Rouen Rendu,417.00
2,2021-02-15,Mais,Rhin Fob,227.00
3,2021-02-15,Mais,Bordeaux Fob,215.00
4,2021-02-15,Ble dur,Port-La-Nouvelle Rendu,240.00
...,...,...,...,...
14323,2024-11-26,Ble tendre,Rouen Rendu,215.50
14324,2024-11-26,Mais,Bordeaux Fob,203.00
14325,2024-11-26,Mais,Rhin Fob,206.75
14326,2024-11-26,Ble tendre,Creil Fob,213.00


In [26]:
dfHist = dfHist.sort_values(by='Date')
dfHist

,Date,Produit,Place,Prix
0,2021-02-15,Mais,Bordeaux Rendu,210.00
13,2021-02-15,Ble tendre,Creil Fob,214.00
12,2021-02-15,Ble tendre,Rouen Fob,278.54
11,2021-02-15,Ble tendre,Rouen Rendu,197.00
9,2021-02-15,Mais,La Pallice Rendu,211.00
...,...,...,...,...
14323,2024-11-26,Ble tendre,Rouen Rendu,215.50
434,2024-11-27,Ble tendre,Eure-et-Loir-Loiret Départ,208.00
215,2024-11-27,Mais,Eure-et-Loir-Loiret Départ,197.00
216,2024-11-28,Mais,Eure-et-Loir-Loiret Départ,194.00


In [28]:
dbname = db.get_database()
collection_name = dbname["new_physique"]
data = dfHist.to_dict('records')
last_doc = collection_name.find_one({})
if last_doc is None:
    r = collection_name.insert_many(data)
else:
    r = 'Impossible de push les données historiques quand il y a déjà des données en base.'
print(r)